In [1]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


# Assumptions for this Approach:
To interwine the least with the provided AIRBUS flight data we decided not to interpolat or impute the missing values for the FUEL_USED in other MSN, because this would be a too big infulence into the model. Also non of us is a aerospace engineer and therefore can not assume how the Fuel consumption should behave and what a  normal behaviour looks like. Therefore in this approach we discard every NAN in the most important columns (`VALUE_FOB, FUEL_USED_1 - FUEL_USED_4`)
  - Not using 1 and 12 Flight phase (because Engine is turned off)
  - Discard all NAN's in `VALUE_FOB`, `FUEL_USED_1 - FUEL_USED_4`
  - Create smoothed values for :
      -  gaussian filtered with sigma = 5: `VALUE_FOB` and `FUEL_USED_1-4`
      -  median filtered with kernel = 5:`VALUE_FOB`
      - interplation polynominal degree = 5:`VALUE_FOB`
  - Creating the Features
  - Create balanced
  - Finding stable Flights where consumption is similar 5th degree polynomial interpolation `VALUE_FOB`
  - then do Scaling for LSTM
  - Split into 80/20 train test
  - Create the sequences 5 s
  - Train on LSTM TO BE CONTINUED...
  

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
msn02= pd.read_csv("/content/drive/MyDrive/IE/CORPORATE_PROJECT_AIRBUS/01_DATA/msn_02_fuel_leak_signals_preprocessed.csv", delimiter=';')
msn10= pd.read_csv("/content/drive/MyDrive/IE/CORPORATE_PROJECT_AIRBUS/01_DATA/msn_02_fuel_leak_signals_preprocessed.csv", delimiter=';')
msn11= pd.read_csv("/content/drive/MyDrive/IE/CORPORATE_PROJECT_AIRBUS/01_DATA/msn_02_fuel_leak_signals_preprocessed.csv", delimiter=';')
msn12= pd.read_csv("/content/drive/MyDrive/IE/CORPORATE_PROJECT_AIRBUS/01_DATA/msn_02_fuel_leak_signals_preprocessed.csv", delimiter=';')
msn14= pd.read_csv("/content/drive/MyDrive/IE/CORPORATE_PROJECT_AIRBUS/01_DATA/msn_02_fuel_leak_signals_preprocessed.csv", delimiter=';')
msn29= pd.read_csv("/content/drive/MyDrive/IE/CORPORATE_PROJECT_AIRBUS/01_DATA/msn_02_fuel_leak_signals_preprocessed.csv", delimiter=';')
msn37= pd.read_csv("/content/drive/MyDrive/IE/CORPORATE_PROJECT_AIRBUS/01_DATA/msn_02_fuel_leak_signals_preprocessed.csv", delimiter=';')
msn53= pd.read_csv("/content/drive/MyDrive/IE/CORPORATE_PROJECT_AIRBUS/01_DATA/msn_02_fuel_leak_signals_preprocessed.csv", delimiter=';')

In [3]:
msn02.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 623580 entries, 0 to 623579
Data columns (total 111 columns):
 #    Column                                Dtype  
---   ------                                -----  
 0    UTC_TIME                              object 
 1    MSN                                   object 
 2    Flight                                object 
 3    ENGINE_RUNNING_1                      bool   
 4    ENGINE_RUNNING_2                      bool   
 5    ENGINE_RUNNING_3                      bool   
 6    ENGINE_RUNNING_4                      bool   
 7    FLIGHT_PHASE_COUNT                    float64
 8    FUEL_FLOW_1                           float64
 9    FUEL_FLOW_2                           float64
 10   FUEL_FLOW_3                           float64
 11   FUEL_FLOW_4                           float64
 12   FUEL_PITCH                            float64
 13   FUEL_ROLL                             float64
 14   FUEL_TRANSFER_MODE_VALUE              float64
 15 

In [4]:
msn53.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 623580 entries, 0 to 623579
Data columns (total 111 columns):
 #    Column                                Dtype  
---   ------                                -----  
 0    UTC_TIME                              object 
 1    MSN                                   object 
 2    Flight                                object 
 3    ENGINE_RUNNING_1                      bool   
 4    ENGINE_RUNNING_2                      bool   
 5    ENGINE_RUNNING_3                      bool   
 6    ENGINE_RUNNING_4                      bool   
 7    FLIGHT_PHASE_COUNT                    float64
 8    FUEL_FLOW_1                           float64
 9    FUEL_FLOW_2                           float64
 10   FUEL_FLOW_3                           float64
 11   FUEL_FLOW_4                           float64
 12   FUEL_PITCH                            float64
 13   FUEL_ROLL                             float64
 14   FUEL_TRANSFER_MODE_VALUE              float64
 15 

## Concatinating all the MSN's on same columns

In [10]:
# List of columns you want to keep for concatenation
columns_to_keep = ["UTC_TIME", "MSN", "Flight","FLIGHT_PHASE_COUNT", "FUEL_USED_1", "FUEL_USED_2", "FUEL_USED_3", "FUEL_USED_4", "VALUE_FOB"]

# List all DataFrames
df_list = [msn02, msn10, msn11, msn12, msn14, msn29, msn37, msn53]

# Concatenate the DataFrames using only the specified columns
# and excluding rows where FLIGHT_PHASE_COUNT is 1 or 12
final_df = pd.concat([df[df['FLIGHT_PHASE_COUNT'].isin([1, 12]) == False][columns_to_keep] for df in df_list], ignore_index=True)



In [11]:
# List all DataFrames
df_list = [msn02, msn10, msn11, msn12, msn14, msn29, msn37, msn53]

# Identify common columns across all DataFrames
common_columns = set.intersection(*(set(df.columns) for df in df_list))

# Exclude boolean columns and keep the order of columns as in msn02
common_columns_ordered = [col for col in msn02.columns if col in common_columns and msn02[col].dtype != 'bool']

# Filter each DataFrame for rows where FLIGHT_PHASE_COUNT is not 1 or 12 and concatenate them
final_df = pd.concat([df.loc[df['FLIGHT_PHASE_COUNT'].isin([1, 12]) == False, common_columns_ordered] for df in df_list], ignore_index=True)


KeyboardInterrupt: 

In [12]:
final_df.to_csv('/content/drive/My Drive/IE/CORPORATE_PROJECT_AIRBUS/01_DATA/concatinated_msn.csv', index=False)

#Cleaning

In [13]:
df= pd.read_csv('/content/drive/My Drive/IE/CORPORATE_PROJECT_AIRBUS/01_DATA/concatinated_msn.csv')

In [14]:
msn = df[df["FLIGHT_PHASE_COUNT"]==8]

In [15]:
msn.shape

(3141640, 9)

In [16]:
# Convert 'UTC_TIME' to datetime
msn['UTC_TIME'] = pd.to_datetime(msn['UTC_TIME'])

# Create new columns for date and time
msn['DATE'] = msn['UTC_TIME'].dt.date
msn['TIME'] = msn['UTC_TIME'].dt.time

<ipython-input-16-bcc257ed4421>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  msn['UTC_TIME'] = pd.to_datetime(msn['UTC_TIME'])
<ipython-input-16-bcc257ed4421>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  msn['DATE'] = msn['UTC_TIME'].dt.date
<ipython-input-16-bcc257ed4421>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [17]:
# Sort the DataFrame by 'UTC_TIME'
msn = msn.sort_values(by='UTC_TIME')

# Reset the index of the DataFrame after sorting
msn = msn.reset_index(drop=True)

In [18]:
msn = msn.dropna(subset=['VALUE_FOB'])
msn = msn.dropna(subset=['FUEL_USED_1'])
msn = msn.dropna(subset=['FUEL_USED_2'])
msn = msn.dropna(subset=['FUEL_USED_3'])
msn = msn.dropna(subset=['FUEL_USED_4'])

In [19]:
# Calculate percentage of NaNs for each column in msn02
nan_percentage_msn = msn.isnull().mean() * 100

# Display the percentage of NaNs for each column in msn02
print(nan_percentage_msn)


UTC_TIME              0.0
MSN                   0.0
Flight                0.0
FLIGHT_PHASE_COUNT    0.0
FUEL_USED_1           0.0
FUEL_USED_2           0.0
FUEL_USED_3           0.0
FUEL_USED_4           0.0
VALUE_FOB             0.0
DATE                  0.0
TIME                  0.0
dtype: float64


In [20]:
# Let's say you want to drop columns 'Column1' and 'Column2'
columns_to_drop = ['APU_FUEL_FLOW_REQUEST_SIGNAL_1', 'TRANSFER_MODE']
msn = msn.drop(columns=columns_to_drop)


KeyError: "['APU_FUEL_FLOW_REQUEST_SIGNAL_1', 'TRANSFER_MODE'] not found in axis"

In [21]:
msn.shape

(3141640, 11)

In [22]:
# Check for NaN values in the specified columns and get the associated FLIGHT_PHASE_COUNT values
nan_phases_fob = msn[msn['VALUE_FOB'].isna()]['FLIGHT_PHASE_COUNT'].unique()
nan_phases_fuel_used_1 = msn[msn['FUEL_USED_1'].isna()]['FLIGHT_PHASE_COUNT'].unique()
nan_phases_fuel_used_2 = msn[msn['FUEL_USED_2'].isna()]['FLIGHT_PHASE_COUNT'].unique()
nan_phases_fuel_used_3 = msn[msn['FUEL_USED_3'].isna()]['FLIGHT_PHASE_COUNT'].unique()
nan_phases_fuel_used_4 = msn[msn['FUEL_USED_4'].isna()]['FLIGHT_PHASE_COUNT'].unique()

# Combine all unique phases with NaN values across the columns into a single set for uniqueness
all_nan_phases = set(nan_phases_fob) | set(nan_phases_fuel_used_1) | set(nan_phases_fuel_used_2) | set(nan_phases_fuel_used_3) | set(nan_phases_fuel_used_4)

# Display the unique flight phases with NaN values
all_nan_phases

set()

In [23]:
from scipy.signal import medfilt

msn['VALUE_FOB_median_filtered'] = medfilt(msn['VALUE_FOB'], kernel_size=5)


In [24]:
msn["VALUE_FOB_polynominal"]= msn["VALUE_FOB"].interpolate(method="polynomial",order=5 )

In [25]:
from scipy.ndimage import gaussian_filter1d

# Define the sigma for Gaussian smoothing
sigma = 5  # Adjust this value based on your specific needs

# List of columns to be smoothed
columns_to_smooth = ['VALUE_FOB',"FUEL_USED_1","FUEL_USED_2","FUEL_USED_3","FUEL_USED_4"]

# Apply Gaussian filter to each column and create a new smoothed column
for col in columns_to_smooth:
    msn[f'{col}_smoothed'] = gaussian_filter1d(msn[col], sigma)

    # Visualize the smoothed versus original data for one of the columns, e.g., 'VALUE_FOB'
    #plt.figure(figsize=(15, 7))
    #plt.plot(msn.index, msn[col], label='Original Interpolated', alpha=0.5)
    #plt.plot(msn.index, msn[f'{col}_smoothed'], label='Smoothed with Gaussian Filter', alpha=0.7)
    #plt.legend()
    #plt.title(f'Gaussian Filter Applied to {col}')
    #plt.xlabel('Index')
    #plt.ylabel(col)
    #plt.show()

# Feature Engineering

1. we sum the `FUEL_USED_1` - `FUEL_USED_4` rowwise. Those columns show th amount of fuel in **Kg/s** used for each engine. so this sum shows the *consumption of fuel* in **kg/s** for the aircraft.

$$
\text{FUEL_USED_TOTAL} (t) = \sum_{i=1}^{4} \text{FUEL\_USED\_i} (t)
$$

2. We calculate the change of consumption of fuel in kg/s

$$
\text{FUEL_USED_TOTAL_change} (t) = \text{FUEL_USED_TOTAL} (t) - \text{FUEL_USED_TOTAL} (t - 1)
$$

3. And now we did the sum of the change of consumption of fuel to get the Amount of Fuel in Kg up until a point of time t

$$
\text{FUEL_USED_TOTAL_cumulative} (t) = \sum_{i=1}^{t} \text{FUEL_USED_TOTAL_change} (t=i)
$$

4. Now we calculated the expected/estimated Fuel on Board in Kg based on the first value of `VALUE_FOB` at **t=0 ** - the amount of Fuel used in kg up until a point of time t (`FUEL_USED_TOTAL_cumulative`)

$$
\text{FOB_EST} (t) =  \text{VALUE_FOB}(t=0) -  \text{FUEL_USED_TOTAL_change} (t=i)
$$

In [26]:

# Calculating the sum of fuel used and total fuel quantity for msn02
msn["FUEL_USED_TOTAL"] = msn["FUEL_USED_1"] + msn["FUEL_USED_2"] + msn["FUEL_USED_3"] + msn["FUEL_USED_4"]
msn["FUEL_USED_TOTAL_smoothed"] = (msn["FUEL_USED_1_smoothed"] + msn["FUEL_USED_2_smoothed"] +msn["FUEL_USED_3_smoothed"] + msn["FUEL_USED_4_smoothed"])


msn["FUEL_USED_TOTAL_change"]=msn["FUEL_USED_TOTAL"].diff().abs().fillna(0)
msn["FUEL_USED_TOTAL_change_smoothed"]=msn["FUEL_USED_TOTAL_smoothed"].diff().abs().fillna(0)


msn['Time_diff_seconds'] = msn.groupby('Flight')['UTC_TIME'].diff().dt.total_seconds().fillna(0)


msn['FUEL_USED_TOTAL_cumulative'] = msn['FUEL_USED_TOTAL_change'].cumsum().abs()
msn['FUEL_USED_TOTAL_cumulative_smoothed'] = msn['FUEL_USED_TOTAL_change_smoothed'].cumsum().abs()


In [27]:
# Group by 'Flight' and transform 'VALUE_FOB' to get the first value for each flight
msn['VALUE_FOB_first'] = msn.groupby("Flight")['VALUE_FOB'].transform('first')
msn['VALUE_FOB_smoothed_first'] = msn.groupby('Flight')['VALUE_FOB_smoothed'].transform('first')
msn["VALUE_FOB_median_filtered_first"]=msn.groupby('Flight')['VALUE_FOB_median_filtered'].transform('first')

# Recalculate FOB_EST using the first VALUE_FOB of each flight
msn["FOB_EST"] = msn["VALUE_FOB_first"] - msn["FUEL_USED_TOTAL_cumulative_smoothed"]
msn["FOB_EST_smoothed"] = msn["VALUE_FOB_smoothed_first"] - msn["FUEL_USED_TOTAL_cumulative_smoothed"]
msn["FOB_EST_median_filtered"] = msn["VALUE_FOB_median_filtered_first"] - msn["FUEL_USED_TOTAL_cumulative_smoothed"]


5. Now we creataed the Difference between estimated Fuel on Board and the measured Fuel on Board

$$
\text{LEAKAGE} (t) =  \text{FOB_EST}(t) -  \text{VALUE_FOB} (t)
$$

 --> so if the estimated *Fuel on Board is higher then the actual FUEL on Board* the Error **should be positiv** - indicating a possible `LEAKAGE`, BUT we can only confirm this as leakage if the error is staying consistently positive and is growing linearly.


6. To understand the changes/growth of that error we also created an Leakage change

$$
\text{LEAKAGE_change} (t) =  \text{LEAKAGE}(t) -  \text{LEAKAGE} (t-1)
$$

In [28]:

msn["LEAKAGE"] =msn["FOB_EST"] - msn["VALUE_FOB"]
msn["LEAKAGE_change"]= msn["LEAKAGE"].diff().abs().fillna(0)

# Calculate LEAKAGE_smoothed by subtracting VALUE_FOB_smoothed from FOB_EST_smoothed
msn["LEAKAGE_smoothed"] = msn["FOB_EST_smoothed"] -msn["VALUE_FOB_smoothed"]
msn["LEAKAGE_smoothed_change"]= msn["LEAKAGE_smoothed"].diff().abs().fillna(0)

msn["LEAKAGE_median_filtered"] =msn["FOB_EST_median_filtered"]- msn["VALUE_FOB_median_filtered"]
msn["LEAKAGE_median_filtered_change"]=msn["LEAKAGE_median_filtered"].diff().abs().fillna(0)

In [29]:
msn["Flight"].unique()

array(['V0094', 'V0095', 'V0097', 'V0098', 'V0099', 'V0132', 'V0133',
       'V0134', 'V0135', 'V0136', 'V0137', 'V0138', 'V0140', 'V0344',
       'V0345', 'V0346', 'V0347', 'V0348', 'V0622', 'V0623', 'V0624',
       'V0625', 'V0626', 'V0833', 'V0834', 'V0835', 'V0836', 'V0837',
       'V0884', 'V0886', 'V0889', 'V0890', 'V0925', 'V0926', 'V0927',
       'V0928', 'V0929', 'V0962', 'V0963', 'V0964', 'V0965', 'V0966'],
      dtype=object)

# Create Leakages

In [31]:
import pandas as pd
import numpy as np

# Assuming msn is your DataFrame containing flight data.

# Define leakage rates and prepare to iterate through them
leakage_rates = [1, 0.3, 0.5, 0.8]

# Initialize the start date for artificial flights to be a day after the last UTC_TIME in msn
last_utc = pd.to_datetime(msn['UTC_TIME'].max())
start_date = last_utc + pd.Timedelta(days=1)

# Store all new flights here
new_flights = []

# For each leakage rate, select 10 random flights and create artificial versions
for leakage_rate in leakage_rates:
    # Reset start_sequence for each batch of 10 flights to ensure unique naming
    start_sequence = 1
    random_flights = np.random.choice(msn['Flight'].unique(), 10, replace=False)

    for flight_index, flight in enumerate(random_flights, start=1):
        flight_data = msn[msn['Flight'] == flight].copy()

        # Ensure we're not altering the original DataFrame index
        flight_data.reset_index(drop=True, inplace=True)

        new_flight_name = f"Artificial_V{start_sequence + flight_index:04d}_{leakage_rate}"
        flight_data['Flight'] = new_flight_name

        # Create the new column by copying VALUE_FOB
        flight_data['VALUE_FOB_with_leak'] = flight_data['VALUE_FOB']
        flight_data['VALUE_FOB_with_leak_smoothed'] = flight_data['VALUE_FOB_smoothed']
        flight_data["VALUE_FOB_with_leak_median_filtered"] = flight_data['VALUE_FOB_median_filtered']

        # Shift UTC_TIME for the artificial flight
        flight_data['UTC_TIME'] = pd.date_range(start=start_date, periods=len(flight_data), freq='S')

        # Identify valid start points for leakage at flight phase 8
        valid_start_points = flight_data[flight_data['FLIGHT_PHASE_COUNT'] == 8].index

        if not valid_start_points.empty:
            leakage_start_point = np.random.choice(valid_start_points)
            print(f"Leakage starts at index {leakage_start_point} for flight {new_flight_name} with rate {leakage_rate}")

            # Apply the extra fuel loss per second after the leakage start point
            for i in range(leakage_start_point, len(flight_data)):
                flight_data.at[i, 'VALUE_FOB'] -= leakage_rate
                flight_data.at[i, 'VALUE_FOB_smoothed'] -= leakage_rate
                flight_data.at[i, 'VALUE_FOB_median_filtered'] -= leakage_rate

        # Update start_date for the next artificial flight
        start_date = flight_data['UTC_TIME'].iloc[-1] + pd.Timedelta(days=1)

        # Append modified flight_data to new_flights
        new_flights.append(flight_data)

        # Increment start_sequence for unique naming in the next batch
        start_sequence += 10

# Combine all new flights into a single DataFrame
new_flights_df = pd.concat(new_flights)


Leakage starts at index 30379 for flight Artificial_V0002_1 with rate 1
Leakage starts at index 50983 for flight Artificial_V0013_1 with rate 1
Leakage starts at index 10051 for flight Artificial_V0024_1 with rate 1
Leakage starts at index 52583 for flight Artificial_V0035_1 with rate 1
Leakage starts at index 8567 for flight Artificial_V0046_1 with rate 1
Leakage starts at index 21674 for flight Artificial_V0057_1 with rate 1
Leakage starts at index 3961 for flight Artificial_V0068_1 with rate 1
Leakage starts at index 35974 for flight Artificial_V0079_1 with rate 1
Leakage starts at index 2577 for flight Artificial_V0090_1 with rate 1
Leakage starts at index 28698 for flight Artificial_V0101_1 with rate 1
Leakage starts at index 16734 for flight Artificial_V0002_0.3 with rate 0.3
Leakage starts at index 43530 for flight Artificial_V0013_0.3 with rate 0.3
Leakage starts at index 64353 for flight Artificial_V0024_0.3 with rate 0.3
Leakage starts at index 12957 for flight Artificial_V00

In [32]:
new_flights_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2594264 entries, 0 to 36487
Data columns (total 40 columns):
 #   Column                               Dtype         
---  ------                               -----         
 0   UTC_TIME                             datetime64[ns]
 1   MSN                                  object        
 2   Flight                               object        
 3   FLIGHT_PHASE_COUNT                   float64       
 4   FUEL_USED_1                          float64       
 5   FUEL_USED_2                          float64       
 6   FUEL_USED_3                          float64       
 7   FUEL_USED_4                          float64       
 8   VALUE_FOB                            float64       
 9   DATE                                 object        
 10  TIME                                 object        
 11  VALUE_FOB_median_filtered            float64       
 12  VALUE_FOB_polynominal                float64       
 13  VALUE_FOB_smoothed           

In [33]:
# Group by 'Flight' and transform 'VALUE_FOB' to get the first value for each flight
new_flights_df['VALUE_FOB_first'] = new_flights_df.groupby("Flight")['VALUE_FOB'].transform('first')
new_flights_df['VALUE_FOB_smoothed_first'] = new_flights_df.groupby('Flight')['VALUE_FOB_smoothed'].transform('first')
new_flights_df["VALUE_FOB_median_filtered_first"]=new_flights_df.groupby('Flight')['VALUE_FOB_median_filtered'].transform('first')

# Recalculate FOB_EST using the first VALUE_FOB of each flight
new_flights_df["FOB_EST"] = new_flights_df["VALUE_FOB_first"] - new_flights_df["FUEL_USED_TOTAL_cumulative_smoothed"]
new_flights_df["FOB_EST_smoothed"] = new_flights_df["VALUE_FOB_smoothed_first"] - new_flights_df["FUEL_USED_TOTAL_cumulative_smoothed"]
new_flights_df["FOB_EST_median_filtered"] = new_flights_df["VALUE_FOB_median_filtered_first"] - new_flights_df["FUEL_USED_TOTAL_cumulative_smoothed"]


In [34]:
new_flights_df["LEAKAGE"] =new_flights_df["FOB_EST"] - new_flights_df["VALUE_FOB"]
new_flights_df["LEAKAGE_change"]= new_flights_df["LEAKAGE"].diff().abs().fillna(0)

# Calculate LEAKAGE_smoothed by subtracting VALUE_FOB_smoothed from FOB_EST_smoothed
new_flights_df["LEAKAGE_smoothed"] = new_flights_df["FOB_EST_smoothed"] -new_flights_df["VALUE_FOB_smoothed"]
new_flights_df["LEAKAGE_smoothed_change"]= new_flights_df["LEAKAGE_smoothed"].diff().abs().fillna(0)

new_flights_df["LEAKAGE_median_filtered"] =new_flights_df["FOB_EST_median_filtered"]- new_flights_df["VALUE_FOB_median_filtered"]
new_flights_df["LEAKAGE_median_filtered_change"]=new_flights_df["LEAKAGE_median_filtered"].diff().abs().fillna(0)

In [35]:
msn_with_leak = pd.concat([msn, new_flights_df], ignore_index=True)

In [36]:
msn_with_leak.to_csv('/content/drive/My Drive/IE/CORPORATE_PROJECT_AIRBUS/01_DATA/msn_all_with_leaks.csv', index=False)

# LSTM

In [2]:
msn= pd.read_csv('/content/drive/My Drive/IE/CORPORATE_PROJECT_AIRBUS/01_DATA/msn_all_with_leaks.csv')

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping


# Step 1: Normalize the features
scaler = StandardScaler()
data_scaled = scaler.fit_transform(msn[['LEAKAGE_smoothed']].values)

# Step 2: Define sequence length and function to create sequences
sequence_length = 5  # Define the length of your sequences

def create_sequences(data, sequence_length):
    X = []
    for i in range(len(data) - sequence_length + 1):
        X.append(data[i:(i + sequence_length)])
    return np.array(X)

X_scaled = create_sequences(data_scaled, sequence_length)

# Step 3: Create labels based on the original 'LEAKAGE_smoothed' data
data_original = msn[['LEAKAGE_smoothed']].values
X_original = create_sequences(data_original, sequence_length)

def label_sequences_based_on_original(X_original):
    y = []
    for sequence in X_original:
        differences = np.diff(sequence, axis=0)
        if np.all(differences > 0):  # All differences are positive
            y.append(1)  # Label as leakage
        else:
            y.append(0)  # Label as no leakage
    return np.array(y)

y = label_sequences_based_on_original(X_original)

# Step 4: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Reshape input to be [samples, time steps, features] which is required for LSTM
X_train = X_train.reshape((X_train.shape[0], sequence_length, 1))
X_test = X_test.reshape((X_test.shape[0], sequence_length, 1))

# Step 5: Define the LSTM model
model = Sequential([
    LSTM(50, activation='relu', input_shape=(sequence_length, 1), return_sequences=True),
    Dropout(0.5),
    LSTM(50, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Step 6: Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 7: Train the model with early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
model.fit(X_train, y_train, epochs=10, batch_size=128, validation_split=0.1, callbacks=[early_stopping])

# Step 8: Evaluate the model
evaluation = model.evaluate(X_test, y_test)
print(f'Test Loss: {evaluation[0]}, Test Accuracy: {evaluation[1]}')


Epoch 1/10
32265/32265 [==============================] - 687s 21ms/step - loss: 0.6845 - accuracy: 0.5655 - val_loss: 0.6845 - val_accuracy: 0.5650
Epoch 2/10
32265/32265 [==============================] - 679s 21ms/step - loss: 0.6844 - accuracy: 0.5655 - val_loss: 0.6845 - val_accuracy: 0.5650
Epoch 3/10
32265/32265 [==============================] - 679s 21ms/step - loss: 0.6844 - accuracy: 0.5655 - val_loss: 0.6845 - val_accuracy: 0.5650
Epoch 4/10
32265/32265 [==============================] - 696s 22ms/step - loss: 0.6844 - accuracy: 0.5655 - val_loss: 0.6845 - val_accuracy: 0.5650
Epoch 5/10
18625/32265 [================>.............] - ETA: 4:51 - loss: 0.6844 - accuracy: 0.5656

KeyboardInterrupt: 

In [4]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming 'final_df' is your DataFrame and it contains a 'LEAKAGE_smoothed' column
# Normalize the features
scaler = MinMaxScaler()
msn['LEAKAGE_smoothed_scaled'] = scaler.fit_transform(msn[['LEAKAGE_smoothed']])

# Create sequences of 5-second windows
sequence_length = 5
def create_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length):
        sequences.append(data[i:i + sequence_length])
    return np.array(sequences)

sequences = create_sequences(msn['LEAKAGE_smoothed_scaled'].values, sequence_length)

# Split the data into training and testing sets
X_train, X_test = train_test_split(sequences, test_size=0.2, random_state=42)

# Reshape input for LSTM [samples, time steps, features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(sequence_length, 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, X_train, epochs=100, batch_size=80)

# Evaluate the model on the testing set
loss = model.evaluate(X_test, X_test)

# Use the model to predict the test set
predicted_leakage = model.predict(X_test)

# Identify anomalies (leaks)
# Assuming that an anomaly is where the predicted value significantly deviates from the actual value
anomaly_indices = []
threshold = 0.01  # This threshold can be adjusted based on your specific requirements
for i in range(len(X_test)):
    if np.abs(predicted_leakage[i] - X_test[i]) > threshold:
        anomaly_indices.append(i)

# anomaly_indices now contains the indices of the anomalies in the test set


Epoch 1/100
33279/57359 [================>.............] - ETA: 2:38 - loss: 7.2814e-04

KeyboardInterrupt: 

### Correct Splits Considering Time Gaps

1. **Segment Data by Flight**: Treat each flight as an independent sequence. This approach ensures that breaks in `UTC_TIME` between flights don't confuse the model by mixing data from separate flights.

2. **Time-Based Splits**: When splitting data into training and testing sets, do so based on time rather than random shuffling. Ensure that the model is trained on earlier flights and tested on later ones to mimic real-world prediction scenarios and to handle time gaps naturally.

3. **Handling Variable-Length Sequences**: If using LSTM and each flight is treated as a sequence, you may encounter variable-length sequences. You can handle these by padding shorter sequences or by designing your model to handle variable lengths explicitly.

### Providing More Inputs to the Neural Network

1. **Feature Engineering**: Beyond the difference between expected and current FOB, consider other relevant features that could indicate leakage or normal operation patterns. Potential features include:
   - Rate of change of FOB over time.
   - Flight phase-specific features (e.g., specific phases might be more prone to showing discrepancies in case of leaks).
   - Engine parameters, weather conditions, or other operational parameters if available.

2. **Multi-Channel Input**: If adding features that have different scales or represent different aspects of the flight, consider using a multi-channel approach where different sets of features are processed by different parts of the model before being combined for the final prediction. This can be achieved by using multiple input layers in Keras.

3. **Temporal Features**: Include temporal features that capture patterns over time, such as rolling averages, lag features, or time since the last maintenance check.

### Tackling the Problem - Step by Step

1. **Data Preparation**:
   - Segment your data by flight.
   - For each flight, generate a set of features and label sequences as having a leak or not based on your criteria.
   - Ensure features are normalized or standardized as appropriate.

2. **Model Input**:
   - Design your model input to include multiple features per time step. For example, each time step might include the current FOB, the expected FOB, the rate of change of FOB, and any other engineered features.

3. **Model Architecture**:
   - Consider an architecture that can handle multiple inputs if you're using different types of data. This could involve parallel LSTM branches for different feature sets or a combination of LSTM and Dense layers for time-series and static features, respectively.

4. **Training and Validation**:
   - Split your data in a time-based manner, ensuring the model is validated on flights that occur after those in the training set.
   - Consider using techniques like cross-validation, but ensure that time-based splitting is respected to prevent data leakage between training and validation sets.

5. **Evaluation**:
   - Evaluate your model based on its ability to accurately identify flights with leaks. Metrics such as precision, recall, and F1 score can be more informative than accuracy, especially if the dataset is imbalanced.

6. **Iteration**:
   - Based on initial results, iterate on feature engineering, model architecture, and data preprocessing. Sometimes, insights gained from initial models can guide improvements in data handling and feature selection.

This structured approach allows you to systematically address the problem, starting from data preparation to model evaluation, while considering the specific challenges of your dataset.
